<a href="https://colab.research.google.com/github/smritiirani/Dcluttr-task/blob/main/SQL1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# STEP 1: Import libraries
import pandas as pd
import sqlite3

# STEP 2: Read uploaded CSV files
scraping_df = pd.read_csv('all_blinkit_category_scraping_stream.csv')
categories_df = pd.read_csv('blinkit_categories.csv')
city_map_df = pd.read_csv('blinkit_city_map.csv')

# STEP 3: Rename columns if needed
scraping_df = scraping_df.rename(columns={
    'created_at': 'scraped_at',
    'l1_category_id': 'category_id',
    'l2_category_id': 'subcategory_id',
    'selling_price': 'price'
})

# STEP 4: Create SQLite DB and store the tables
con = sqlite3.connect('blinkit.db')
scraping_df.to_sql('all_blinkit_category_scraping_stream', con, if_exists='replace', index=False)
categories_df.to_sql('blinkit_categories', con, if_exists='replace', index=False)
city_map_df.to_sql('blinkit_city_map', con, if_exists='replace', index=False)

# STEP 5: SQL Query to generate the insights table
query = """
WITH movement AS (
    SELECT
        s.store_id, s.sku_id, s.scraped_at,
        s.category_id, s.subcategory_id, s.brand_id,
        s.brand, s.image_url, s.sku_name,
        s.price, s.mrp, s.inventory,
        LAG(s.inventory) OVER (
            PARTITION BY s.store_id, s.sku_id ORDER BY s.scraped_at
        ) AS prev_inventory
    FROM all_blinkit_category_scraping_stream s
),
estimation AS (
    SELECT
        store_id, sku_id, scraped_at,
        category_id, subcategory_id, brand_id,
        brand, image_url, sku_name, price, mrp,
        CASE
            WHEN prev_inventory IS NOT NULL AND inventory < prev_inventory THEN prev_inventory - inventory
            ELSE 0
        END AS est_qty_sold,
        inventory
    FROM movement
),
joined AS (
    SELECT
        e.scraped_at as date, e.sku_id, e.store_id,
        e.sku_name, e.brand_id, e.brand, e.image_url,
        e.price, e.mrp, e.inventory, e.est_qty_sold,
        c.l1_category_id AS category_id,
        c.l1_category AS category_name,
        c.l2_category_id AS sub_category_id,
        c.l2_category AS sub_category_name,
        m.city_name
    FROM estimation e
    JOIN blinkit_categories c
      ON e.category_id = c.l1_category_id AND e.subcategory_id = c.l2_category_id
    JOIN blinkit_city_map m
      ON e.store_id = m.store_id
)
SELECT
    date,
    sku_id,
    city_name AS city,
    sku_name,
    brand_id,
    brand,
    image_url,
    category_id,
    category_name,
    sub_category_id,
    sub_category_name,
    SUM(est_qty_sold) AS est_qty_sold,
    ROUND(SUM(est_qty_sold * price), 2) AS est_sales_sp,
    ROUND(SUM(est_qty_sold * mrp), 2) AS est_sales_mrp
FROM joined
GROUP BY date, city, category_name, sub_category_name, sku_id
ORDER BY date, city, category_name
"""

# STEP 6: Run the query and save results
result_df = pd.read_sql_query(query, con)
result_df.to_csv("blinkit_city_insights.csv", index=False)

# STEP 7: Download the file
from google.colab import files
files.download("blinkit_city_insights.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>